<a href="https://colab.research.google.com/github/sgu20191619/DACON/blob/main/%EC%A0%9C%EC%A3%BC%EB%8F%84_%EA%B5%90%ED%86%B5%EB%9F%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import gc

In [ ]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [ ]:
csv_to_parquet('/content/drive/MyDrive/open (1)/train.csv', 'train')
csv_to_parquet('/content/drive/MyDrive/open (1)/test.csv', 'test')
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

train Done.
test Done.


In [ ]:
def get_outlier(df=None, column=None, weight=1.25):
  # target 값과 상관관계가 높은 열을 우선적으로 진행
  quantile_25 = np.percentile(df[column].values, 25)
  quantile_75 = np.percentile(df[column].values, 75)

  IQR = quantile_75 - quantile_25
  IQR_weight = IQR*weight
  
  lowest = quantile_25 - IQR_weight
  highest = quantile_75 + IQR_weight
  
  outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
  return outlier_idx

outlier_idx = get_outlier(df=train, column='target', weight=1.5)
train.drop(outlier_idx, axis=0, inplace=True)

In [ ]:
def get_peak(x) :
  if x >= 20210918 and x <= 20210923 :
    return 1
  elif x == 20211231 : 
    return 1
  elif x == 20220101 :
    return 1
  elif x >= 20220129 and x <= 20220203 :
    return 1
  elif x >= 20220220 and x <= 20220301 :
    return 1
  elif x >= 20220504 and x <= 20220508 :
    return 1
  elif x >= 20220604 and x <= 20220606 :
    return 1
  elif x >= 20220723 and x <= 20220821 :
    return 1
  elif x >= 20220908 and x <= 20220913 :
    return 1
  elif x >= 20221001 and x <= 20221003 :
    return 1
  elif x == 20221231 :
    return 1
  else : return 0  

train["peak"] = train["base_date"].apply(lambda x :get_peak(x))
test["peak"] = test["base_date"].apply(lambda x : get_peak(x))

In [ ]:
train['month'] = train['base_date'].mod(10000)
test['month'] = test['base_date'].mod(10000)
train['month'] = train['month'].where(train['month'] < 100 ,train['month'] // 100)
test['month'] = test['month'].where(test['month'] < 100 ,test['month'] // 100)
# train에는 8월이 없기때문에 7월처럼 생각해줌 
test.loc[test['month'] ==  8, 'month'] = 7

In [ ]:
# 제주특별자치도관광협회(http://visitjeju.or.kr/web/bbs/bbsList.do?bbsId=TOURSTAT)
# 2021년 8~12월, 2022년 1~7월 데이터 사용

def tour(x) :
    if x == 1 :
        return 1170802
    elif x == 2 :
        return 1029503
    elif x == 3 :
        return 873086
    elif x == 4 :
        return 1178456
    elif x == 5 :
        return 1306537
    elif x == 6 :
        return 1283470
    elif x == 7 or x == 8:
        return 1263332
    elif x == 9 :
        return 872396
    elif x == 10 :
        return 1222094
    elif x == 11 :
        return 1204344
    else :
        return 1090607

train["tourist"] = train["month"].apply(lambda x :tour(x))
test["tourist"] = test["month"].apply(lambda x : tour(x) * 1.25)

In [ ]:
# 제주특별자치도-주민등록인구통계(https://www.jeju.go.kr/open/stats/list/population.htm?year=2021)
# 2021년 8~12월, 2022년 1~7월 데이터 사용
# 월에 0.01퍼 정도 상승

def reside(x) :
    if x == 1 :
        return 697269
    elif x == 2 :
        return 697718
    elif x == 3 :
        return 697841
    elif x == 4 :
        return 698056
    elif x == 5 :
        return 698435
    elif x == 6 :
        return 698698
    elif x == 7 :
        return 699303
    elif x == 8 :
        return 697108
    elif x == 9 :
        return 697263
    elif x == 10 :
        return 697647
    elif x == 11 :
        return 697718
    else :
        return 697476

train["resident"] = train["month"].apply(lambda x :reside(x))
test["resident"] = test["month"].apply(lambda x : reside(x) * 1.013)

In [ ]:
def mode_hour(x) :
    if x == 1 :
        return 51.5
    elif x == 2 :
        return 51.5
    elif x == 3 :
        return 52.5
    elif x == 4 :
        return 52.5
    elif x == 5 :
        return 51.5
    elif x == 6 :
        return 50.5
    elif x == 7 :
        return 46.4
    elif x == 8 :
        return 48
    elif x == 9 :
        return 41.5
    elif x == 10 :
        return 36.1
    elif x == 11 :
        return 38.1
    elif x == 12 :
        return 35.8
    elif x == 13 :
        return 38
    elif x == 14 :
        return 38.1
    elif x == 15 :
        return 36.9
    elif x == 16 :
        return 35.9
    elif x == 17 :
        return 30.6
    elif x == 18 :
        return 36.1
    elif x == 19 :
        return 37.1
    elif x == 20 :
        return 39.6
    elif x == 21 :
        return 42.8
    elif x == 22 :
        return 48.7
    elif x == 23 :
        return 49.5
    else :
        return 51.5

train["modest_hour"] = train["base_hour"].apply(lambda x : mode_hour(x))
test["modest_hour"] = test["base_hour"].apply(lambda x : mode_hour(x))
# train['mod_week_hour'] = train.apply(lambda x: x['modest_hour'] * (x['day_of_week']+5), axis = 1)
# test['mod_week_hour'] = test.apply(lambda x: x['modest_hour'] * (x['day_of_week']+5), axis = 1)

In [ ]:
missing = train.loc[train['road_rating'].isin([107]) & train['road_name'].isin(['-'])]
miss_test = test.loc[test['road_rating'].isin([107]) & test['road_name'].isin(['-'])] 
index = missing.index
index2 = miss_test.index
train_missing = train.loc[train['road_rating'].isin([107])].drop(index)
test_missing = test.loc[test['road_rating'].isin([107])].drop(index2)
road = dict()
all = []
for s,e,r in zip(train_missing['start_node_name'],train_missing['end_node_name'], train_missing['road_name']) :
  all.append((s,e))
  if r not in road :
    road[r] = []
  if (s,e) not in road[r] :
    road[r].append((s,e)) 
all = set(all)

miss = []
cnt = 0 
for s,e in zip(missing['start_node_name'], missing['end_node_name']) :
  miss.append((s,e))
miss = set(miss)

miss2 = []
cnt = 0 
for num,s,e in zip(index2, miss_test['start_node_name'], miss_test['end_node_name']) :
  miss2.append((s,e))
  for j in road : 
    if (s,e) in road[j] :
      test.iloc[num]['road_name'] = road[j]
      print(test.iloc[num]['road_name'])
    if (e,s) in road[j] :
      test.iloc[num]['road_name'] = road[j]
      print(test.iloc[num]['road_name'])
      cnt = cnt + 1
miss2 = set(miss2)

# train.loc[train['road_name'] == '-', 'road_name' ] = np.nan
# test.loc[test['road_name'] == '-', 'road_name'] = np.nan

In [ ]:
from haversine import haversine

In [ ]:
data = train[['start_latitude','start_longitude']]
start = [tuple(x) for x in data.values]
data = train[['end_latitude','end_longitude']]
end = [tuple(x) for x in data.values]

In [ ]:
distance = []
for i in range(len(start)) :
 distance.append(haversine(start[i], end[i]))
train['distance'] = distance 

data = test[['start_latitude','start_longitude']]
start = [tuple(x) for x in data.values]
data = test[['end_latitude','end_longitude']]
end = [tuple(x) for x in data.values]

distance = []
for i in range(len(start)) :
  distance.append(haversine(start[i], end[i]))

test['distance'] = distance 


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#from catboost import CatBoostRegressor

In [ ]:
train = train.drop(columns = (['road_in_use']))
test = test.drop(columns = (['road_in_use']))

In [ ]:
X = train.drop(columns = ['id','target','height_restricted', 'vehicle_restricted', 'multi_linked','connect_code', 'maximum_speed_limit','start_node_name', 'end_node_name','base_date'], axis = 1)
y = train['target'] 
test_x = test.drop(['id', 'height_restricted', 'vehicle_restricted','multi_linked','connect_code','maximum_speed_limit','start_node_name', 'end_node_name','base_date'], axis=1)

In [ ]:
# X = X.drop(['road_name'], axis = 1)
# test_x = test_x.drop(['road_name'],axis = 1)

In [ ]:
str_col = ['day_of_week', 'road_name']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(X[i])
    X[i]=le.transform(X[i])
    print(i)
    for label in np.unique(test_x[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i]=le.transform(test_x[i])

day_of_week
road_name


In [ ]:
# X['distance'] = X['distance'] * 100 

In [ ]:
X['weight_restricted'] = X['weight_restricted'] * 0.0001

In [ ]:
test_x['weight_restricted'] = test_x['weight_restricted'] * 0.0001

In [ ]:
# 6               road_type        8040
# 12    end_turn_restricted        9615
# 9   start_turn_restricted       11629
# 5       weight_restricted       13121

X = X.drop(columns = ['road_type', 'end_turn_restricted', 'start_turn_restricted','weight_restricted'], axis = 1)
test_x = test_x.drop(columns = ['road_type', 'end_turn_restricted', 'start_turn_restricted','weight_restricted'], axis = 1)

In [ ]:
# X['mod_week_hour'] = X.apply(lambda x: x['modest_hour'] * (x['day_of_week']+5), axis = 1)
# test_x['mod_week_hour'] = test_x.apply(lambda x: x['modest_hour'] * (x['day_of_week']+5), axis = 1)

In [ ]:
!pip install optuna

In [ ]:
from sklearn.metrics import mean_absolute_error
def MAE(y, y_pred):
  mae = mean_absolute_error(y, y_pred)
  return mae

import optuna
import lightgbm as lgb
from optuna.samplers import TPESampler
cut = int(len(train)*0.8)
h_train_X = X[:cut]
h_valid_X = X[cut:]

h_train_y = y[:cut]
h_valid_y = y[cut:]
sampler = TPESampler(seed = 10)

def objective(trial):
    dtrain = lgb.Dataset(h_train_X, label=h_train_y)
    dtest = lgb.Dataset(h_valid_X, label=h_valid_y)

    param = {
        'objective': 'regression',
        'verbose': -1,
        'metric': 'mae', 
        'max_depth': trial.suggest_int('max_depth',10, 100),
        'learning_rate': trial.suggest_loguniform("learning_rate", 0.05, 0.13),
        'n_estimators': trial.suggest_int('n_estimators', 4000, 8000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 120),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
        'num_leaves' : trial.suggest_int('num_leaves', 1000, 100000),
        'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 15, 30)
    }

    model = lgb.LGBMRegressor(**param)
    lgb_model = model.fit(h_train_X, h_train_y, eval_set=[(h_valid_X, h_valid_y)], verbose=0, early_stopping_rounds=25)
    mae = MAE(h_valid_y, lgb_model.predict(h_valid_X))
    return mae

study_lgb = optuna.create_study(direction='minimize', sampler=sampler)
study_lgb.optimize(objective, n_trials=100)

In [ ]:
X=X.drop(columns = ['mod_week_hour'],axis=1)
test_x= test_x.drop(columns = ['mod_week_hour'],axis=1)
X.head()

,day_of_week,base_hour,lane_count,road_rating,road_name,start_latitude,start_longitude,end_latitude,end_longitude,peak,month,tourist,resident,modest_hour,distance
0,1,17,1,106,46,33.427747,126.662612,33.427749,126.662335,0,6,1283470,698698,30.6,0.025711
1,1,21,2,103,34,33.500730,126.529107,33.504811,126.526240,1,7,1263332,699303,42.8,0.525891
2,4,7,2,103,36,33.279145,126.368598,33.280072,126.362147,0,10,1222094,697647,46.4,0.608399
3,0,13,2,107,57,33.246081,126.567204,33.245565,126.566228,0,3,873086,697841,38.0,0.107352
4,6,8,2,103,35,33.462214,126.326551,33.462677,126.330152,0,10,1222094,697647,48.0,0.337949


In [ ]:
# # LGBM - KFold
from sklearn.model_selection import KFold
import lightgbm as lgb

k_fold = KFold(n_splits = 3, shuffle=True, random_state = 123)
y_pred = np.zeros(len(test))

for train_idx, val_idx in k_fold.split(X):
  x_t = X.iloc[train_idx]
  y_t = y.iloc[train_idx]
  x_val = X.iloc[val_idx]
  y_val = y.iloc[val_idx]
  
  lgbm = lgb.LGBMRegressor(n_estimators = 7000,
                           learning_rate = 0.05,
                           max_depth = 40,
                           num_leaves = 6800,
                           min_data_in_leaf = 20,
                           min_child_samples = 250,
                           #subsample = 0.85,
                           metric = 'mae',
                           objective = 'regression')
  lgbm.fit(x_t, y_t, eval_set=[(x_val, y_val)], eval_metric= 'mae', early_stopping_rounds= 25,verbose = 1)
  y_pred += lgbm.predict(test_x) / 3

[1]	valid_0's l1: 12.6146
Training until validation scores don't improve for 25 rounds.
[2]	valid_0's l1: 12.0358
[3]	valid_0's l1: 11.4884
[4]	valid_0's l1: 10.9706
[5]	valid_0's l1: 10.4813
[6]	valid_0's l1: 10.0191
[7]	valid_0's l1: 9.58265
[8]	valid_0's l1: 9.17048
[9]	valid_0's l1: 8.78173
[10]	valid_0's l1: 8.41531
[11]	valid_0's l1: 8.06987
[12]	valid_0's l1: 7.74457
[13]	valid_0's l1: 7.43831
[14]	valid_0's l1: 7.15018
[15]	valid_0's l1: 6.87935
[16]	valid_0's l1: 6.62511
[17]	valid_0's l1: 6.38631
[18]	valid_0's l1: 6.1624
[19]	valid_0's l1: 5.95234
[20]	valid_0's l1: 5.75568
[21]	valid_0's l1: 5.57152
[22]	valid_0's l1: 5.39906
[23]	valid_0's l1: 5.23784
[24]	valid_0's l1: 5.08701
[25]	valid_0's l1: 4.94657
[26]	valid_0's l1: 4.81536
[27]	valid_0's l1: 4.69295
[28]	valid_0's l1: 4.57883
[29]	valid_0's l1: 4.47278
[30]	valid_0's l1: 4.37382
[31]	valid_0's l1: 4.28174
[32]	valid_0's l1: 4.19621
[33]	valid_0's l1: 4.11668
[34]	valid_0's l1: 4.04273
[35]	valid_0's l1: 3.97404
[36

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/open (1)/sample_submission.csv')

In [ ]:
submission['target'] = y_pred.round(0)

In [ ]:
submission.to_csv('./submit.csv', index = False)

In [ ]:
from lightgbm import plot_importance

In [ ]:
submission

,id,target
0,TEST_000000,26.0
1,TEST_000001,42.0
2,TEST_000002,69.0
3,TEST_000003,39.0
4,TEST_000004,43.0
...,...,...
291236,TEST_291236,47.0
291237,TEST_291237,51.0
291238,TEST_291238,22.0
291239,TEST_291239,23.0


In [ ]:
# 특성 중요도 시각화 하기
import plotly.express as px

feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': lgbm.feature_importances_
})

feature_importance.sort_values('importance', inplace=True)
print(feature_importance)

fig = px.bar(feature_importance, x='feature', y='importance')
fig.show()

            feature  importance
3       road_rating       26279
2        lane_count       49214
9              peak       54762
12         resident       56532
4         road_name       70641
13      modest_hour      103974
11          tourist      107890
14         distance      128925
8     end_longitude      139612
6   start_longitude      178970
7      end_latitude      179429
10            month      218040
5    start_latitude      258687
0       day_of_week      341015
1         base_hour      418087
